In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm

In [2]:
url = "https://www.imdb.com/list/ls068082370/"

In [3]:
def all_page_link(start_url):
    all_urls = []
    url = start_url
    while(url != None):            #Loop around all the required webpages and terminates when last page arive!
        all_urls.append(url)
        soup = BeautifulSoup(requests.get(url).text,"html.parser")
        next_links = soup.find_all(class_='flat-button lister-page-next next-page')    #Extracts the next page link.
        if (len(next_links) == 0):         # If their is no next page, it returns 0.
            url = None
        else:
            next_page = "https://www.imdb.com" + next_links[0].get('href')
            url = next_page
    return all_urls

In [4]:
def director_and_actor(Director_and_star):
    Director_and_star =  Director_and_star.replace("\n","")
    Director_and_star = Director_and_star.replace("|","")
    Director_and_star = Director_and_star.split("Stars:")
    Director_and_star[0] = Director_and_star[0].replace("Director:","")
    Director_and_star[0] = Director_and_star[0].replace("Directors:","")
    for i in range(10):
        Director_and_star[0]=Director_and_star[0].replace("  "," ")
    director = Director_and_star[0][1:]
    stars = Director_and_star[1]
    stars = stars.replace(":","")
    return director,stars

In [5]:
def votes_and_gross_conveter(votes_and_gross):
    votes_and_gross_list = []
    for i in votes_and_gross:
        votes_and_gross_list.append(i.text)
    if(len(votes_and_gross)==2):
        votes=votes_and_gross_list[0]
        gross = votes_and_gross_list[1]
    else:
        votes=votes_and_gross_list[0]
        gross = None
    
    return votes,gross

In [6]:
main_array = []
for url in tqdm(all_page_link("https://www.imdb.com/list/ls068082370/")):     #Runs the function for all the pages.
    soup = BeautifulSoup(requests.get(url).text,"html.parser")         #Extracts out the main html code.
    for link in soup.find_all(class_='lister-item-content'):
        id = int(link.find('span',{"class":"lister-item-index unbold text-primary"}).text[:-1])
        name = link.find('a').text
        year = link.find('span',{"class":"lister-item-year text-muted unbold"}).text[1:5]
        run_time = link.find('span',{"class":"runtime"}).text
        genre = link.find('span',{"class":"genre"}).text[1:]
        rating = link.find('span',{"class":"ipl-rating-star__rating"}).text
        about = link.find_all('p')[1].text[5:]
        director,actors = director_and_actor(link.find_all('p',{"class":"text-muted text-small"})[1].text)
        votes, gross = votes_and_gross_conveter(link.find_all('span',{"name":"nv"}))
        votes = int(votes.replace(",",""))
        list_of_all = [id,name,year,run_time,genre,rating,about,director,actors,votes,gross]
        main_array.append(list_of_all)

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:21<00:00,  7.29s/it]


In [7]:
index = ["id","name","year","run_time","genre","rating","about","director","actors","votes","gross"]

In [8]:
df = pd.DataFrame(main_array,columns=index)

In [9]:
df.head(5)

,id,name,year,run_time,genre,rating,about,director,actors,votes,gross
0,1,The Shawshank Redemption,1994,142 min,Drama,9.3,"imprisoned men bond over a number of years, fi...",Frank Darabont,"Tim Robbins, Morgan Freeman, Bob Gunton, Willi...",2451775,$28.34M
1,2,The Godfather,1972,175 min,"Crime, Drama",9.2,rganized crime dynasty's aging patriarch trans...,Francis Ford Coppola,"Marlon Brando, Al Pacino, James Caan, Diane Ke...",1696937,$134.97M
2,3,The Dark Knight,2008,152 min,"Action, Crime, Drama",9,the menace known as the Joker wreaks havoc an...,Christopher Nolan,"Christian Bale, Heath Ledger, Aaron Eckhart, M...",2407467,$534.86M
3,4,The Godfather: Part II,1974,202 min,"Crime, Drama",9,early life and career of Vito Corleone in 1920...,Francis Ford Coppola,"Al Pacino, Robert De Niro, Robert Duvall, Dian...",1178645,$57.30M
4,5,Pulp Fiction,1994,154 min,"Crime, Drama",8.9,"lives of two mob hitmen, a boxer, a gangster a...",Quentin Tarantino,"John Travolta, Uma Thurman, Samuel L. Jackson,...",1900380,$107.93M


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        250 non-null    int64 
 1   name      250 non-null    object
 2   year      250 non-null    object
 3   run_time  250 non-null    object
 4   genre     250 non-null    object
 5   rating    250 non-null    object
 6   about     250 non-null    object
 7   director  250 non-null    object
 8   actors    250 non-null    object
 9   votes     250 non-null    int64 
 10  gross     224 non-null    object
dtypes: int64(2), object(9)
memory usage: 21.6+ KB


In [11]:
df['about']

0      imprisoned men bond over a number of years, fi...
1      rganized crime dynasty's aging patriarch trans...
2       the menace known as the Joker wreaks havoc an...
3      early life and career of Vito Corleone in 1920...
4      lives of two mob hitmen, a boxer, a gangster a...
                             ...                        
245    ior and pacifist Princess Nausicaä desperately...
246    ivate detective takes on a case that involves ...
247    rse is put in charge of a mute actress and fin...
248    or Midwest family is forced off their land. Th...
249     a killer shark unleashes chaos on a beach com...
Name: about, Length: 250, dtype: object

In [12]:
df.to_csv('first_250.csv')